# Wahaha train


まとめてインポート

In [1]:
!pip install pytorch-lightning
!pip install transformers

     |████████████████████████████████| 526 kB 4.2 MB/s 
     |████████████████████████████████| 596 kB 67.2 MB/s 
     |████████████████████████████████| 132 kB 68.6 MB/s 
     |████████████████████████████████| 332 kB 71.8 MB/s 
     |████████████████████████████████| 829 kB 67.1 MB/s 
     |████████████████████████████████| 1.1 MB 56.5 MB/s 
     |████████████████████████████████| 160 kB 72.1 MB/s 
     |████████████████████████████████| 271 kB 63.9 MB/s 
     |████████████████████████████████| 192 kB 82.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=c5f60c9c25544698133621372a76fd4e4a9496dd3acc14faa886b54a75ea2e38
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found exis

In [1]:
!nvidia-smi

Wed Jan  5 16:24:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import pathlib
import json
import glob
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from transformers import AdamW
from transformers import BertConfig, BertForMaskedLM, BertModel, BertForSequenceClassification
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint


In [2]:

# TRAIN_LOG_DIRECTORY='drive/MyDrive/dl/wahaha/log'
TRAIN_LOG_DIRECTORY='drive/MyDrive/dl/wahaha/log'
TRAIN_MLM_MODEL_DIR = 'drive/MyDrive/dl/wahaha/model'
TRAIN_PRETRAIN_MODEL_BIN = 'drive/MyDrive/dl/wahaha/model/pretrain_mlm/pytorch_model.bin'

TRAIN_FILE='drive/MyDrive/dl/wahaha/train/train_feature.txt'
TEST_FILE='drive/MyDrive/dl/wahaha/train/test_feature.txt'
# TRAIN_DATA_DIRECTORY='drive/MyDrive/dl/kanachan/train'
# TEST_DATA_DIRECTORY='drive/MyDrive/dl/kanachan/test'

TRAIN_POSSIBLE_LABEL_COUNT = 219 # label count

TRAIN_TOKEN_CLS     = 0
TRAIN_TOKEN_SEP     = 1
TRAIN_TOKEN_PADDING = 2 + 382 + 648 + 219 # 2(CLS,SEP)+382(sparse)+648(progression)+219(possible)
TRAIN_TOKEN_MASK    = TRAIN_TOKEN_PADDING + 1

TRAIN_TOKEN_VOCAB_COUNT = TRAIN_TOKEN_PADDING + 2 # 2(PADDING, MASK)
TRAIN_MAX_TOKEN_LENGTH = 2 + 139 + 1 # 26(sparse)+81(progression)+32(possible)
TRAIN_ID_LIST = list(range(2, TRAIN_TOKEN_PADDING))

TRAIN_HIDDEN_SIZE = 768

In [3]:
# https://github.com/PyTorchLightning/pytorch-lightning/issues/2534#issuecomment-674582085
class CheckpointEveryNSteps(pl.Callback):
    """
    Save a checkpoint every N steps, instead of Lightning's default that checkpoints
    based on validation loss.
    """

    def __init__(
        self,
        save_step_frequency,
        prefix="N-Step-Checkpoint",
        use_modelcheckpoint_filename=False,
    ):
        """
        Args:
            save_step_frequency: how often to save in steps
            prefix: add a prefix to the name, only used if
                use_modelcheckpoint_filename=False
            use_modelcheckpoint_filename: just use the ModelCheckpoint callback's
                default filename, don't use ours.
        """
        self.save_step_frequency = save_step_frequency
        self.prefix = prefix
        self.use_modelcheckpoint_filename = use_modelcheckpoint_filename

    def on_batch_end(self, trainer: pl.Trainer, _):
        """ Check if we should save a checkpoint after every train batch """
        epoch = trainer.current_epoch
        global_step = trainer.global_step
        if global_step % self.save_step_frequency == 0:
            if self.use_modelcheckpoint_filename:
                filename = trainer.checkpoint_callback.filename
            else:
                #filename = f"{self.prefix}_{epoch}_{global_step}.ckpt"
                filename = f"{self.prefix}.ckpt"
            ckpt_path = os.path.join(TRAIN_LOG_DIRECTORY, filename)
            trainer.save_checkpoint(ckpt_path)


## MLM(Masked Language Model)
MLMによる事前学習を実施します。 

In [ ]:
TRAIN_MLM_CONFIG = {
    'vocab_size': TRAIN_TOKEN_VOCAB_COUNT,  # MASK_TOKEN_ID, MASK, CLS, SEP
    'hidden_size': 768,
    'num_hidden_layers': 12,
    'num_attention_heads': 12,
    'intermediate_size': 3072,  # hidden_size * 4が目安
    'hidden_act': 'gelu',
    'hidden_dropout_prob': 0.1,
    'attention_probs_dropout_prob': 0.1,
    'max_position_embeddings': 512,  # 95(=81(マス目)+7(先手持駒)+7(後手持駒))でいいかも
    'type_vocab_size': 1,  # 対の文章を入れない。つまりtoken_type_embeddingsは完全に無駄になっている。
    'initializer_range': 0.02,
}

TRAIN_MLM_MODEL_CONFIG = {
    'checkpoint_callback' : False, 
    'gpus' : [0], # 環境にあわせて変更
    'val_check_interval' : 10000, # validationおよびcheckpointの間隔step数
    'max_epochs' : 5,
    'model_dir' : None, # インポートするモデル
}
TRAIN_MLM_LOADER_CONFIG = {
    'batch_size' : 64,
    'num_workers' : 8,
    'pin_memory' : False,
    'drop_last' : True
}
VAL_MLM_LOADER_CONFIG = {
    'batch_size' : 64,
    'num_workers' : 8,
    'pin_memory' : False,
    'drop_last' : False
}


In [ ]:
config = BertConfig.from_dict(TRAIN_MLM_CONFIG)

class BertMLM(nn.Module):
    def __init__(self, model_dir=None):
        super().__init__()
        if model_dir is None:
            self.bert = BertForMaskedLM(config)
        else:
            self.bert = BertForMaskedLM.from_pretrained(model_dir)

    def forward(self, input_ids, labels):
        return self.bert(input_ids=input_ids, labels=labels)


In [ ]:
class MLMModule(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        model_dir = hparams["model_dir"]
        self.save_hyperparameters(hparams)
        self.model = BertMLM(model_dir)

    def forward(self, batch):
        input_ids = batch['input_ids']
        labels = batch['labels']
        return self.model(input_ids=input_ids, labels=labels)

    def training_step(self, batch, batch_idx):
        outputs = self(batch)
        loss = outputs[0]
        self.log('loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self(batch)
        loss = outputs[0].detach().cpu().numpy()
        return {'loss': loss}

    def validation_epoch_end(self, outputs):
        val_loss = np.mean([out['loss'] for out in outputs])
        #self.log('steps', self.global_step)
        self.log('val_loss', val_loss)

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=5e-5)

class MLMDataset(Dataset):
    def __init__(self, filename):
        self.filename = filename # input text file
        with open(self.filename, 'r') as file :
            print("[MLMDataset]--- start setup ---")
            self.content = file.readlines()
            print("[MLMDataset]--- loading is completed ---")

        self.mask_token_id = TRAIN_TOKEN_MASK  # 駒が割り振られていないid

    def __len__(self):
        return len(self.content)

    def __getitem__(self, idx):
        line = self.content[idx]
        inputs, _ = line.split('\t') # labelは破棄
        inputs = json.loads('[' + inputs + ']')
        inputs = [TRAIN_TOKEN_CLS] + inputs + [TRAIN_TOKEN_SEP]
        
        inputs = np.array(inputs)
        inputs = np.pad(inputs,(0, TRAIN_MAX_TOKEN_LENGTH - len(inputs)), constant_values = TRAIN_TOKEN_PADDING)
        labels = inputs.copy()

        # 予想対象
        masked_indices = np.random.random(labels.shape) < 0.15
        labels[~masked_indices] = -100

        # 80%はマスクトークンに
        indices_replaced = (np.random.random(labels.shape) < 0.8) & masked_indices
        inputs[indices_replaced] = self.mask_token_id

        # 10%はランダムに置き換え
        indices_random = (np.random.random(labels.shape) < 0.5) & masked_indices & ~indices_replaced
        random_words = np.random.choice(TRAIN_ID_LIST, labels.shape)
        inputs[indices_random] = random_words[indices_random]

        # 残り10%はそのままのものが残る
        ret_dict = {'input_ids': torch.tensor(inputs, dtype=torch.long),
                    'labels': torch.tensor(labels, dtype=torch.long)}
        return ret_dict

class MLMDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        # self.cfg = cfg

    def setup(self, stage=None) :
        self.train_dataset = MLMDataset(TRAIN_FILE)
        self.val_dataset = MLMDataset(TEST_FILE)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, **TRAIN_MLM_LOADER_CONFIG)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, **VAL_MLM_LOADER_CONFIG)


In [ ]:
trainer = pl.Trainer(gpus=1, max_epochs=5, enable_checkpointing=False, callbacks=[CheckpointEveryNSteps(save_step_frequency = 10000)])
trainer.logger = pl_loggers.TensorBoardLogger(save_dir=TRAIN_LOG_DIRECTORY, name=TRAIN_NAME, default_hp_metric=False)
trainer.callbacks.append(ModelCheckpoint(filename='{step:07d}-{val_loss:.2f}', monitor='val_loss',
                                             save_top_k=1, save_last=True))
model = MLMModule(TRAIN_MLM_MODEL_CONFIG)
data = MLMDataModule()
trainer.fit(model, datamodule=data)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


[MLMDataset]--- start setup ---
[MLMDataset]--- loading is completed ---
[MLMDataset]--- start setup ---


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[MLMDataset]--- loading is completed ---



  | Name  | Type    | Params
----------------------------------
0 | model | BertMLM | 87.0 M
----------------------------------
87.0 M    Trainable params
0         Non-trainable params
87.0 M    Total params
348.023   Total estimated model params size (MB)


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:686: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


## Classification
BertForSequenceClassificationによる学習

In [ ]:
#!echo "start" > start_test.txt

TRAIN_CLASSIFICATION_CONFIG = {
    'vocab_size': TRAIN_TOKEN_VOCAB_COUNT,  # MASK_TOKEN_ID, MASK, CLS, SEP
    'hidden_size': TRAIN_HIDDEN_SIZE,
    'num_hidden_layers': 12,
    'num_attention_heads': 12,
    'intermediate_size': 3072,  # hidden_size * 4が目安
    'hidden_act': 'gelu',
    'hidden_dropout_prob': 0.1,
    'attention_probs_dropout_prob': 0.1,
    'max_position_embeddings': 512,  # 95(=81(マス目)+7(先手持駒)+7(後手持駒))でいいかも
    'type_vocab_size': 1,  # 対の文章を入れない。つまりtoken_type_embeddingsは完全に無駄になっている。
    'initializer_range': 0.02,
    'num_labels' : TRAIN_POSSIBLE_LABEL_COUNT, 
}
TRAIN_CLASSIFICATION_MODEL_CONFIG = {
    # 'checkpoint_callback' : False, 
    'gpus' : 1, # 環境にあわせて変更
    'val_check_interval' : 10000, # validationおよびcheckpointの間隔step数
    'max_epochs' : 3,
    'model_dir' : TRAIN_MLM_MODEL_DIR, # インポートするモデル
    'enable_checkpointing' : False, 
}

TRAIN_CLASSIFICATION_LOADER_CONFIG = {
    'batch_size' : 64,
    'num_workers' : 2,
    'pin_memory' : True,
    'drop_last' : True
}
VAL_CLASSIFICATION_LOADER_CONFIG = {
    'batch_size' : 64,
    'num_workers' : 2,
    'pin_memory' : True,
    'drop_last' : False
}
BERT_CLASSIFICATION_CONFIG = BertConfig.from_dict(TRAIN_CLASSIFICATION_CONFIG)

class BertClassification(nn.Module) :
    def __init__(self, model_dir):
        super().__init__()
        model_bin = model_dir + "/pytorch_model.bin"
        self.bert = BertForSequenceClassification.from_pretrained(model_bin, config = BERT_CLASSIFICATION_CONFIG)

    def forward(self, input_ids, labels):
        outputs = self.bert(input_ids, labels = labels)
        loss = outputs[0]
        # logit = outputs[1]
        return loss

class BertClassificationModule(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        model_dir = hparams["model_dir"]
        self.save_hyperparameters(hparams)
        self.model = BertClassification(model_dir)

    def forward(self, input_ids, labels):
        return self.model(input_ids=input_ids, labels=labels)

    def training_step(self, batch, batch_idx):
        input_ids = batch.pop('input_ids')
        labels = batch.pop('labels')
        outputs = self(input_ids, labels)
        #loss = outputs['loss']
        return outputs

    def validation_step(self, batch, batch_idx):
        input_ids = batch.pop('input_ids')
        labels = batch.pop('labels')
        output = self(input_ids, labels)
        return output

    def validation_epoch_end(self, outputs):
        #print(outputs)
        #val_loss = np.mean([out for out in outputs])
        val_loss = np.mean([out.to('cpu').detach().numpy() for out in outputs])
        #val_loss = np.mean(outputs.to('cpu').detach().numpy())
        self.log('val_loss', val_loss)

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=5e-5)

class BertClassificationDataset(Dataset):
    def __init__(self, filename):
        # まとめて読み出し
        self.filename = filename
        with open(self.filename, 'r') as file :
            print("[MLMDataset]--- start setup ---")
            self.content = file.readlines()
            print("[MLMDataset]--- loading is completed ---")

    def __len__(self):
        return len(self.content)

    def __getitem__(self, idx):
        line = self.content[idx]
        inputs, label = line.split('\t')
        inputs = json.loads('[' + inputs + ']')
        inputs = [TRAIN_TOKEN_CLS] + inputs + [TRAIN_TOKEN_SEP]
        
        # padding付与
        inputs = np.array(inputs)
        inputs = np.pad(inputs,(0, TRAIN_MAX_TOKEN_LENGTH - len(inputs)), constant_values = TRAIN_TOKEN_PADDING)

        label = [int(label)]

        ret_dict = {'input_ids': torch.tensor(inputs, dtype=torch.long),
                    'labels': torch.tensor(label, dtype=torch.long)}
        return ret_dict

class BertClassificationDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()

    def setup(self, stage=None) :
        self.train_dataset = BertClassificationDataset(TRAIN_FILE)
        self.val_dataset = BertClassificationDataset(TEST_FILE)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, **TRAIN_CLASSIFICATION_LOADER_CONFIG)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, **VAL_CLASSIFICATION_LOADER_CONFIG)

# Train
hparams = TRAIN_CLASSIFICATION_MODEL_CONFIG
trainer = pl.Trainer(gpus = hparams["gpus"], enable_checkpointing=hparams["enable_checkpointing"], max_epochs = hparams["max_epochs"], callbacks = [CheckpointEveryNSteps(save_step_frequency = 10000)])
trainer.logger = pl_loggers.TensorBoardLogger(save_dir=TRAIN_LOG_DIRECTORY, name="wahaha_classification", default_hp_metric=False)
#trainer.callbacks.append(ModelCheckpoint(filename='{step:07d}-{val_loss:.2f}', monitor='val_loss',
#                                             save_top_k=1, save_last=True))
model = BertClassificationModule(TRAIN_CLASSIFICATION_MODEL_CONFIG)
data = BertClassificationDataModule()
trainer.fit(model, datamodule=data)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Some weights of the model checkpoint at drive/MyDrive/dl/wahaha/model/pytorch_model.bin were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassif

[MLMDataset]--- start setup ---
[MLMDataset]--- loading is completed ---
[MLMDataset]--- start setup ---


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[MLMDataset]--- loading is completed ---


Missing logger folder: drive/MyDrive/dl/wahaha/log/wahaha_classification

  | Name  | Type               | Params
---------------------------------------------
0 | model | BertClassification | 87.2 M
---------------------------------------------
87.2 M    Trainable params
0         Non-trainable params
87.2 M    Total params
348.685   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]